In [22]:
openai_key = "sk-wBPEE7XTm2rXiTc4ux4LkFEUfARLpl8zgAisgfja6yT3BlbkFJQs8x0TQMWv2_j-69UEkR5UzNPyxlm9oKbhVsOr5IMA"

In [23]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

from tqdm import tqdm
import bs4
from bs4 import BeautifulSoup
import requests
from docx import Document
from langchain_community.document_loaders import WebBaseLoader

In [24]:
## source document
url = "https://eclinic.mn/patient/blog/"
urls = []
for i in range(1, 5):
    urls.append(f"{url}{i}")

In [25]:
doc = Document()

for url in tqdm(urls):
    print(url)
    r = requests.get(url)
    # Parsing the HTML
    soup = BeautifulSoup(r.content, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)
    doc.add_paragraph(text)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

https://eclinic.mn/patient/blog/1


 25%|█████████████████████                                                               | 1/4 [00:00<00:02,  1.30it/s]

https://eclinic.mn/patient/blog/2


 50%|██████████████████████████████████████████                                          | 2/4 [00:01<00:01,  1.28it/s]

https://eclinic.mn/patient/blog/3


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:02<00:00,  1.18it/s]

https://eclinic.mn/patient/blog/4


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.23it/s]


In [26]:
doc_path = "ecilinic.docx"
doc.save(doc_path)

In [27]:
import re
def preprocess(text):
    text = text.replace(' .', '.')
    text = text.replace(' ,', ',')
    text = text.replace("\n", "")
    text = re.sub('[^A-Za-zА-яӨөҮүЁё0-9!""#$%&()*+,-./;<=>?@^_`\[\]{}:]+', " ", text)
    return text.strip()

In [28]:
len(docs[0].page_content)

15851

## Rag preparation

In [30]:
system_prompt = (
    "You are an assistant of Healthcare for question-answering tasks. "
    "Use the following context to answer the question. If you don't know the answer, say that you "
    "don't know. Use 5 sentences maximum and keep the answer concise."
    "\n\n"
    "{context}")

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.05,
                 max_tokens=None,
                 timeout=None,
                 max_retries=2,
                 api_key=openai_key)

question_answer_chain = create_stuff_documents_chain(llm, prompt)


In [31]:
chunk_size = int((1.2*len(docs[0].page_content))/154)

In [32]:
chunk_size

123

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=int(1.2*chunk_size), 
                                               chunk_overlap=int(0.1*chunk_size),
                                               is_separator_regex=False,  
                                               separators=["?", "!", "\n\n"])
all_splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=all_splits, 
                                    embedding=OpenAIEmbeddings(api_key=openai_key, 
                                                               show_progress_bar=True))
retriever = vectorstore.as_retriever()

  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
from colorama import Fore, Back, Style

In [ ]:
valid_response = False
while not valid_response:
    question = Fore.RED+input("Enter your question: ")
    if question.lower() == 'yes':
        valid_response = True
    else:
        response = rag_chain.invoke({"input": question})
        print(Fore.RED+"Answer: ", response["answer"])

Enter your question:  Үс унаад байна ямар арга хэмжээ авах уу


  0%|          | 0/1 [00:00<?, ?it/s]

Answer:  Үс уналтыг зогсоохын тулд хэд хэдэн арга хэмжээ авах боломжтой. Нэгдүгээрт, арьсны чийгшлийг тогтмол байлгахын тулд чийгшүүлэгч бүтээгдэхүүн хэрэглэх хэрэгтэй. Хоёрдугаарт, илүүдэл тос болон бохирдлыг арилгахын тулд нүүрээ өдөрт 2 удаа угаах нь чухал. Мөн зөөлөн гуужуулагч ашиглах, гарынхаа хүрэлцээг багасгах, батгыг оролдож шахахгүй байх хэрэгтэй. Хэрвээ асуудал үргэлжилбэл мэргэжлийн эмчид хандахыг зөвлөж байна.


Enter your question:  Үс унаад байна ямар арга хэмжээ авахуу дугаарлаад хэлээд өгөөч


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Answer:  Батга гараад байвал нүүрээ өдөрт 2 удаа зөөлөн угааж, хэт их савандаж угаахаас зайлсхийх хэрэгтэй. Арьсны чийглэгийг хадгалахын тулд чийгшүүлэгч бүтээгдэхүүн хэрэглэх нь чухал. Зөөлөн гуужуулагчийг хэрэглэж, батгыг оролдож, шахахгүй байх хэрэгтэй. Мөн нүүрэндээ гараараа хүрэхгүй байхыг анхаараарай. Хэрэв батга үргэлжилбэл эмчид хандах нь зүйтэй.


Enter your question:  үс үналтыг зогсоох 5 арга


  0%|          | 0/1 [00:00<?, ?it/s]

Answer:  Үс уналтыг зогсоохын тулд дараах 5 аргыг хэрэгжүүлж болно:

1. Зөв хооллолт: Витамин, минерал, амин хүчил агуулсан хоол хүнс хэрэглэх.
2. Стрессийг бууруулах: Сэтгэл зүйн стрессийг удирдах арга техникийг ашиглах.
3. Арьс болон үсний арчилгаа: Үсний тос, чийгшүүлэгч бүтээгдэхүүн хэрэглэх.
4. Эмчилгээ: Эмчийн зөвлөгөөгөөр үсний уналтын эмчилгээ хийлгэх.
5. Хэт их халуун, химийн бүтээгдэхүүнээс зайлсхийх: Үсийг хэт их халуунд хатаах, химийн боловсруулалт хийхээс татгалзах.
